In [1]:
import pandas as pd
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
file_path = r'C:\Users\nimri\Downloads\SuicideData - Form Responses 1.csv'
df = pd.read_csv(file_path)
df.fillna('لم أقم بتجربة التنمر', inplace=True)
df.head()

,Timestamp,العمر,الجنس,تاريخ الصحة النفسية,التفكير في الانتحار,الحالة العقلية الحالية,الدعم الاجتماعي,وسائل التكيف,الوصول إلى وسائل انتحارية,البحث عن المساعدة,الأحداث الحياتية,الانعزال عن الآخرين,التجربة مع التنمر,نوع التنمر (إذا تم التعرض له ),التعامل مع التنمر,الإبلاغ عن التنمر,الدعم للتنمر
0,11/3/2023 17:53:26,25-34,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,لا، لم أفكر أبدًا في إيذاء نفسي أو إنهاء حياتي.,إيجابية.,نظام دعم قوي.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لقد قمت بتجربة التنمر في الماضي.,التنمر البدني (على سبيل المثال، الضرب، الدفع).,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لقد تلقيت الدعم من الأصدقاء، العائلة أو المحتر...
1,11/3/2023 17:56:23,18-24,ذكر,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,لا، لم أفكر أبدًا في إيذاء نفسي أو إنهاء حياتي.,إيجابية.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة، وأنا غير مستعد للقيام...,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,التنمر اللفظي (على سبيل المثال، استخدام ألفاظ ...,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
2,11/3/2023 18:01:27,25-34,ذكر,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,لا، لم أفكر أبدًا في إيذاء نفسي أو إنهاء حياتي.,إيجابية جدًا.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,نعم، لدي وصول إلى مواد قاتلة (أدوية )،أدوات ح...,نعم، لقد بحثت عن المساعدة من محترف الصحة النفس...,لم أشهد أحداث حياتية مهمة مؤخرًا.,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,استبعاد اجتماعي .,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
3,11/3/2023 18:12:46,18-24,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,نعم، لقد فكرت في الإيذاء الذاتي أو الانتحار في...,إيجابية.,نظام دعم محدود.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,نعم، لدي وصول إلى مواد قاتلة (أدوية )،أدوات ح...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لم أشهد أحداث حياتية مهمة مؤخرًا.,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لقد قمت بتجربة التنمر في الماضي.,التنمر اللفظي (على سبيل المثال، استخدام ألفاظ ...,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
4,11/3/2023 18:19:51,18-24,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,لا، لم أفكر أبدًا في إيذاء نفسي أو إنهاء حياتي.,إيجابية.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,لم أقم بتجربة التنمر,لدي استراتيجيات فعالة للتعامل مع التنمر.,لقد قمت بالإبلاغ عن حالات التنمر إلى الجهات ال...,لقد تلقيت الدعم من الأصدقاء، العائلة أو المحتر...


In [87]:
columns_to_drop = ['Timestamp']
df = df.drop(columns=columns_to_drop)
print("\nDataFrame after dropping specified columns:")
df.head()


DataFrame after dropping specified columns:


,العمر,الجنس,تاريخ الصحة النفسية,التفكير في الانتحار,الحالة العقلية الحالية,الدعم الاجتماعي,وسائل التكيف,الوصول إلى وسائل انتحارية,البحث عن المساعدة,الأحداث الحياتية,الانعزال عن الآخرين,التجربة مع التنمر,نوع التنمر (إذا تم التعرض له ),التعامل مع التنمر,الإبلاغ عن التنمر,الدعم للتنمر
0,25-34,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,لا، لم أفكر أبدًا في إيذاء نفسي أو إنهاء حياتي.,إيجابية.,نظام دعم قوي.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لقد قمت بتجربة التنمر في الماضي.,التنمر البدني (على سبيل المثال، الضرب، الدفع).,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لقد تلقيت الدعم من الأصدقاء، العائلة أو المحتر...
1,18-24,ذكر,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,لا، لم أفكر أبدًا في إيذاء نفسي أو إنهاء حياتي.,إيجابية.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة، وأنا غير مستعد للقيام...,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,التنمر اللفظي (على سبيل المثال، استخدام ألفاظ ...,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
2,25-34,ذكر,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,لا، لم أفكر أبدًا في إيذاء نفسي أو إنهاء حياتي.,إيجابية جدًا.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,نعم، لدي وصول إلى مواد قاتلة (أدوية )،أدوات ح...,نعم، لقد بحثت عن المساعدة من محترف الصحة النفس...,لم أشهد أحداث حياتية مهمة مؤخرًا.,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,استبعاد اجتماعي .,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
3,18-24,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,نعم، لقد فكرت في الإيذاء الذاتي أو الانتحار في...,إيجابية.,نظام دعم محدود.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,نعم، لدي وصول إلى مواد قاتلة (أدوية )،أدوات ح...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لم أشهد أحداث حياتية مهمة مؤخرًا.,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لقد قمت بتجربة التنمر في الماضي.,التنمر اللفظي (على سبيل المثال، استخدام ألفاظ ...,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
4,18-24,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,لا، لم أفكر أبدًا في إيذاء نفسي أو إنهاء حياتي.,إيجابية.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,لم أقم بتجربة التنمر,لدي استراتيجيات فعالة للتعامل مع التنمر.,لقد قمت بالإبلاغ عن حالات التنمر إلى الجهات ال...,لقد تلقيت الدعم من الأصدقاء، العائلة أو المحتر...


In [88]:
le = LabelEncoder()
df['التفكير في الانتحار'] = le.fit_transform(df['التفكير في الانتحار'])
df.head()

,العمر,الجنس,تاريخ الصحة النفسية,التفكير في الانتحار,الحالة العقلية الحالية,الدعم الاجتماعي,وسائل التكيف,الوصول إلى وسائل انتحارية,البحث عن المساعدة,الأحداث الحياتية,الانعزال عن الآخرين,التجربة مع التنمر,نوع التنمر (إذا تم التعرض له ),التعامل مع التنمر,الإبلاغ عن التنمر,الدعم للتنمر
0,25-34,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,0,إيجابية.,نظام دعم قوي.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لقد قمت بتجربة التنمر في الماضي.,التنمر البدني (على سبيل المثال، الضرب، الدفع).,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لقد تلقيت الدعم من الأصدقاء، العائلة أو المحتر...
1,18-24,ذكر,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,0,إيجابية.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة، وأنا غير مستعد للقيام...,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,التنمر اللفظي (على سبيل المثال، استخدام ألفاظ ...,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
2,25-34,ذكر,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,0,إيجابية جدًا.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,نعم، لدي وصول إلى مواد قاتلة (أدوية )،أدوات ح...,نعم، لقد بحثت عن المساعدة من محترف الصحة النفس...,لم أشهد أحداث حياتية مهمة مؤخرًا.,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,استبعاد اجتماعي .,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
3,18-24,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,2,إيجابية.,نظام دعم محدود.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,نعم، لدي وصول إلى مواد قاتلة (أدوية )،أدوات ح...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لم أشهد أحداث حياتية مهمة مؤخرًا.,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لقد قمت بتجربة التنمر في الماضي.,التنمر اللفظي (على سبيل المثال، استخدام ألفاظ ...,لدي استراتيجيات فعالة للتعامل مع التنمر.,لم أقم بالإبلاغ عن حالات التنمر.,لم أتلق الدعم في التعامل مع التنمر.
4,18-24,أنثى,لا، لم يتم تشخيصي بأي حالة صحية نفسية.,0,إيجابية.,نظام دعم متوسط.,وسائل التكيف الصحية (على سبيل المثال، التحدث م...,لا، ليس لدي وصول إلى مواد قاتلة (أدوية )،أدوات...,لا، لم أبحث عن المساعدة ولكنني مستعد للقيام بذلك.,لقد شهدت أحداث حياتية مهمة مؤخرًا (على سبيل ال...,أنا متصل اجتماعيًا وأتفاعل بانتظام مع الأصدقاء...,لم أقم بتجربة التنمر.,لم أقم بتجربة التنمر,لدي استراتيجيات فعالة للتعامل مع التنمر.,لقد قمت بالإبلاغ عن حالات التنمر إلى الجهات ال...,لقد تلقيت الدعم من الأصدقاء، العائلة أو المحتر...


In [89]:
df = pd.get_dummies(df, columns=['تاريخ الصحة النفسية', 'الحالة العقلية الحالية','الدعم الاجتماعي','وسائل التكيف','الوصول إلى وسائل انتحارية','البحث عن المساعدة','الأحداث الحياتية','الانعزال عن الآخرين','التجربة مع التنمر','نوع التنمر (إذا تم التعرض له )','التعامل مع التنمر','الإبلاغ عن التنمر','الدعم للتنمر'])
, dtype=int)


'dtype=int)'

In [90]:
df.replace({'أنثى': 1, 'ذكر': 0}, inplace=True)     
df.head()

,العمر,الجنس,التفكير في الانتحار,تاريخ الصحة النفسية_لا، لم يتم تشخيصي بأي حالة صحية نفسية.,تاريخ الصحة النفسية_نعم، تم تشخيصي بحالة صحية نفسية، لكني لا أتلقى علاجًا حاليًا.,تاريخ الصحة النفسية_نعم، تم تشخيصي بحالة صحية نفسية، وأنا حاليًا أتلقى علاجًا.,الحالة العقلية الحالية_إيجابية جدًا.,الحالة العقلية الحالية_إيجابية.,الحالة العقلية الحالية_سلبية جدًا.,الحالة العقلية الحالية_سلبية.,...,التعامل مع التنمر_لدي استراتيجيات فعالة للتعامل مع التنمر.,التعامل مع التنمر_لم أقم بتجربة التنمر,التعامل مع التنمر_لم أقم بتجربة التنمر,الإبلاغ عن التنمر_لقد قمت بالإبلاغ عن حالات التنمر إلى الجهات المعنية (مثل المدرسة أو مكان العمل).,الإبلاغ عن التنمر_لم أقم بالإبلاغ عن حالات التنمر.,الإبلاغ عن التنمر_لم أقم بتجربة التنمر,الإبلاغ عن التنمر_لم أقم بتجربة التنمر,الدعم للتنمر_لقد تلقيت الدعم من الأصدقاء، العائلة أو المحترفين في التعامل مع التنمر.,الدعم للتنمر_لم أتلق الدعم في التعامل مع التنمر.,الدعم للتنمر_لم أقم بتجربة التنمر
0,25-34,1,0,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,1,0,0
1,18-24,0,0,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0
2,25-34,0,0,1,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
3,18-24,1,2,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0
4,18-24,1,0,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,0


In [91]:
#Decision Tree Classifier
label_encoder = LabelEncoder()
df['التفكير في الانتحار'] = label_encoder.fit_transform(df['التفكير في الانتحار'])
df = pd.get_dummies(df, dtype=int)

X = df.drop('التفكير في الانتحار', axis=1)
y = df['التفكير في الانتحار']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

dt_classifier = DecisionTreeClassifier()
# 10 Fold Cross-Validation
y_pred_cv = cross_val_predict(dt_classifier, X_resampled, y_resampled, cv=10)
conf_matrix_cv = confusion_matrix(y_resampled, y_pred_cv)
accuracy_cv = accuracy_score(y_resampled, y_pred_cv)

print("Confusion Matrix (10-fold CV with SMOTE):\n", conf_matrix_cv)
print("Accuracy (10-fold CV with SMOTE):", accuracy_cv)
#Hold-out 
dt_classifier.fit(X_train, y_train)
y_pred_holdout = dt_classifier.predict(X_test)
conf_matrix_holdout = confusion_matrix(y_test, y_pred_holdout)
accuracy_holdout = accuracy_score(y_test, y_pred_holdout)

print("\nConfusion Matrix (Hold-out Method):\n", conf_matrix_holdout)
print("Accuracy (Hold-out Method):", accuracy_holdout)


Confusion Matrix (10-fold CV with SMOTE):
 [[460  23 158]
 [ 15 610  16]
 [132  27 482]]
Accuracy (10-fold CV with SMOTE): 0.8070722828913156

Confusion Matrix (Hold-out Method):
 [[122   2  34]
 [  6   3   3]
 [ 40   5  19]]
Accuracy (Hold-out Method): 0.6153846153846154


In [92]:
#Naive Bayes Classifier
X = df.drop('التفكير في الانتحار', axis=1)
y = df['التفكير في الانتحار']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()
# 10 Fold Cross-Validation
y_pred_cv = cross_val_predict(nb_classifier, X_train, y_train, cv=10)
conf_matrix_cv = confusion_matrix(y_train, y_pred_cv)
accuracy_cv = accuracy_score(y_train, y_pred_cv)
print("Confusion Matrix (10-fold CV with SMOTE):\n", conf_matrix_cv)
print("Accuracy (10-fold CV with SMOTE):", accuracy_cv)
#Hold-out 
nb_classifier.fit(X_train, y_train)
y_pred_holdout = nb_classifier.predict(X_test)
conf_matrix_holdout = confusion_matrix(y_test, y_pred_holdout)
accuracy_holdout = accuracy_score(y_test, y_pred_holdout)
print("\nConfusion Matrix (Hold-out Method):\n", conf_matrix_holdout)
print("Accuracy (Hold-out Method):", accuracy_holdout)


Confusion Matrix (10-fold CV with SMOTE):
 [[ 14  40 576]
 [  0 567  70]
 [  3 135 512]]
Accuracy (10-fold CV with SMOTE): 0.5701617110067815

Confusion Matrix (Hold-out Method):
 [[  6  15 148]
 [  1 148  13]
 [  0  47 102]]
Accuracy (Hold-out Method): 0.5333333333333333


In [93]:
# K-Nearest Neighbors Classifier
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")
X = df.drop('التفكير في الانتحار', axis=1)
y = df['التفكير في الانتحار']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

knn_classifier = KNeighborsClassifier()

# 10-fold Cross-Validation
y_pred_cv = cross_val_predict(knn_classifier, X_train, y_train, cv=10)
conf_matrix_cv = confusion_matrix(y_train, y_pred_cv)
accuracy_cv = accuracy_score(y_train, y_pred_cv)
print("Confusion Matrix (10-fold CV with SMOTE):\n", conf_matrix_cv)
print("Accuracy (10-fold CV with SMOTE):", accuracy_cv)

# Hold-out 
knn_classifier.fit(X_train, y_train)
y_pred_holdout = knn_classifier.predict(X_test)
conf_matrix_holdout = confusion_matrix(y_test, y_pred_holdout)
accuracy_holdout = accuracy_score(y_test, y_pred_holdout)
print("\nConfusion Matrix (Hold-out Method):\n", conf_matrix_holdout)
print("Accuracy (Hold-out Method):", accuracy_holdout)


Confusion Matrix (10-fold CV with SMOTE):
 [[428  31 171]
 [  1 635   1]
 [ 98  42 510]]
Accuracy (10-fold CV with SMOTE): 0.8205529473135107

Confusion Matrix (Hold-out Method):
 [[126   7  36]
 [  0 162   0]
 [ 19  11 119]]
Accuracy (Hold-out Method): 0.8479166666666667


In [94]:
# Support Vector Machine (SVM) Classifier

X = df.drop('التفكير في الانتحار', axis=1)
y = df['التفكير في الانتحار']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

svm_classifier = SVC()

# 10-fold Cross-Validation
y_pred_cv = cross_val_predict(svm_classifier, X_train, y_train, cv=10)
conf_matrix_cv = confusion_matrix(y_train, y_pred_cv)
accuracy_cv = accuracy_score(y_train, y_pred_cv)
print("Confusion Matrix (10-fold CV with SMOTE):\n", conf_matrix_cv)
print("Accuracy (10-fold CV with SMOTE):", accuracy_cv)

# Hold-out
svm_classifier.fit(X_train, y_train)
y_pred_holdout = svm_classifier.predict(X_test)
conf_matrix_holdout = confusion_matrix(y_test, y_pred_holdout)
accuracy_holdout = accuracy_score(y_test, y_pred_holdout)
print("\nConfusion Matrix (Hold-out Method):\n", conf_matrix_holdout)
print("Accuracy (Hold-out Method):", accuracy_holdout)


Confusion Matrix (10-fold CV with SMOTE):
 [[557  10  63]
 [ 12 617   8]
 [164  36 450]]
Accuracy (10-fold CV with SMOTE): 0.8471570161711007

Confusion Matrix (Hold-out Method):
 [[154   3  12]
 [  2 158   2]
 [ 38  10 101]]
Accuracy (Hold-out Method): 0.8604166666666667


In [95]:
# Ensemble (Random Forest Classifier)
# Separate features (X) and target variable (y)
X = df.drop('التفكير في الانتحار', axis=1)
y = df['التفكير في الانتحار']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(random_state=42)

# 10-fold Cross-Validation
y_pred_cv = cross_val_predict(rf_classifier, X_train, y_train, cv=10)
conf_matrix_cv = confusion_matrix(y_train, y_pred_cv)
accuracy_cv = accuracy_score(y_train, y_pred_cv)
print("Confusion Matrix (10-fold CV):\n", conf_matrix_cv)
print("Accuracy (10-fold CV):", accuracy_cv)

# Hold-out 
rf_classifier.fit(X_train, y_train)
y_pred_holdout = rf_classifier.predict(X_test)
conf_matrix_holdout = confusion_matrix(y_test, y_pred_holdout)
accuracy_holdout = accuracy_score(y_test, y_pred_holdout)
print("\nConfusion Matrix (Hold-out Method):\n", conf_matrix_holdout)
print("Accuracy (Hold-out Method):", accuracy_holdout)


Confusion Matrix (10-fold CV):
 [[570   5  66]
 [ 24   5  20]
 [163   6  73]]
Accuracy (10-fold CV): 0.6952789699570815

Confusion Matrix (Hold-out Method):
 [[145   0  13]
 [  6   2   4]
 [ 43   1  20]]
Accuracy (Hold-out Method): 0.7136752136752137
